In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import re
import os
%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,5)

data_dir = 'content'

In [32]:
filenames = [filename for filename in os.listdir(data_dir)]

In [33]:
kw_empty = ['1744.dat', '14790.dat', '11173.dat', '11772.dat', '26525.dat', '17774.dat', '7553.dat', '2629.dat',
             '8710.dat', '13645.dat', '1894.dat', '14060.dat', '27040.dat', '8862.dat', '26202.dat', '24173.dat',
             '13222.dat', '22011.dat', '6972.dat', '2268.dat']
kw_broken = ['22948.dat', '20331.dat', '19349.dat', '17777.dat', '18928.dat', '13501.dat', '17392.dat',
            '9978.dat', '14103.dat', '6455.dat', '453.dat', '3952.dat', '20038.dat', '5868.dat', '1210.dat',
            '17929.dat']
not_unique = ['16953.dat']
alone = ['20069.dat','27042.dat', '27737.dat', '26800.dat', '4078.dat', '8569.dat', '11658.dat', '17122.dat', '17485.dat',
         '15838.dat', '10590.dat', '3002.dat', '15566.dat', '18840.dat']

In [34]:
def get_keywords(filename):
    global garb
    if filename == '.DS_Store':
        return
    with open(data_dir + '/' + filename) as f:
        
        dt = f.read().lower()
        
        dt = re.sub(r'\s+', ' ', dt)
        dt = re.sub(r'\&[a-zA-Z]{1,8}\;', ' ', dt)
        
        dt = re.sub(r'<[ ]*br[ ]*>', '', dt)
        dt = re.sub(r'<[ ]*br[ ]*/[ ]*>', '', dt)
        dt = re.sub(r'<em>', '', dt)
        dt = re.sub(r'<\/em>', '', dt)
        dt = re.sub(r'<samp>', '', dt) 
        dt = re.sub(r'<\/div>', '', dt)
        dt = re.sub(r'<[ ]*small[ ]*>', '', dt)
        dt = re.sub(r'<[ ]*strong[ ]*>', '', dt)
        dt = re.sub(r'<[ ]*\/small[ ]*>', '', dt)
        dt = re.sub(r'<[ ]*\/strong[ ]*>', '', dt)
        dt = re.sub(r'<[ ]*p[ ]*>', '', dt)
        dt = re.sub(r'<[ ]*/p[ ]*>', '', dt)
        dt = re.sub(r'<[ ]*b[ ]*>', '', dt)
        dt = re.sub(r'<[ ]*\/b[ ]*>', '', dt)
        dt = re.sub(r'<[ ]*i[ ]*>', '', dt)
        dt = re.sub(r'<[ ]*\/i[ ]*>', '', dt)
        dt = re.sub(r'<[ ]*u[ ]*>', '', dt)
        dt = re.sub(r'<[ ]*\/u[ ]*>', '', dt)
        
        dt = re.sub(r'\[\/img\]', ' ', dt)
        dt = re.sub(r'\[attach\]', ' ', dt)
        
        dt = re.sub(r'\.ru\b', ' ', dt)
        dt = re.sub(r'\.com\b', ' ', dt)
        dt = re.sub(r'\.ua\b', ' ', dt)
        dt = re.sub(r'\.ру\b', ' ', dt)
        
        dt = re.sub(r'\s+lt\s+', ' ', dt)
        dt = re.sub(r'\s+gt\s+', ' ', dt)
        
        dt = re.sub(r'http\:\/\/[\w\-&\./?=\+;@#%]*', ' ', dt)
        dt = re.sub(r'https\:\/\/[\w\-&\./?=\+;@#%]*', ' ', dt)
        dt = re.sub(r'ftp\:\/\/[\w\-&\./?=\+;@#%]*', ' ', dt)
        dt = re.sub(r'www\.[\w\-&\./?=\+;@#%]*', ' ', dt)
            
        t = re.findall(r'<meta [^><]*name[^><]*=[^><]*"keywords[^>]*>', dt)
        
        with open('keywords/{}_kw.txt'.format(filename), 'w') as out:
            if len(t) != 0:
                if filename in kw_broken:
                    tt = re.findall(r'content[^$]*', t[0])
                    if len(tt) > 0:
                        tt = tt[0]
                        tt = re.sub('content', '', tt)
                        print(tt, file=out)
                    else:
                        print('', file=out)
                        
                elif filename in alone:
                    t = re.findall('[а-я]+', dt)
                    tt = ''
                    bord = int(0.8 * len(t))
                    t = t[bord:bord + 100]
                    for st in t:
                        tt += st + ' '
                    print(tt, file=out)
                    
                elif filename not in kw_empty:
                    if filename in not_unique:
                        t[0] = t[1] 
                    tt1 = re.findall(r'content[ ]*=[ ]*["][^"]*["]', t[0]) 
                    tt2 = re.findall(r'content[ ]*=[ ]*[\'][^\']*[\']', t[0]) 
                    tt3 = re.findall(r'pluginent[ ]*=[ ]*["][^"]*["]', t[0]) 
                    tt4 = re.findall(r'pluginent[ ]*=[ ]*[\'][^\']*[\']', t[0])
                    desc = [tt1, tt2, tt3, tt4]
                    non_zero_ind = np.argmax(np.array([len(tt1), len(tt2), len(tt3), len(tt4)]))
                    tt = desc[non_zero_ind][0]
                    tt = re.sub('content[ ]*=', '', tt)
                    tt = re.sub('pluginent[ ]*=', '', tt)
                    tt = tt.strip()
                    print(tt, file=out)
                    
                else:
                    print('', file=out)

In [35]:
from multiprocessing.dummy import Pool, Queue
import os 
data_dir = 'content' 

queue = Queue()
for filename in filenames:
    queue.put(filename)
    
queue.empty()

False

In [36]:
def process_page_wrapper(i):
    while not queue.empty():
        filename = queue.get()
        get_keywords(filename)

        with lock:
            pbar.update(1)
                
with Pool(processes=4) as pool, tqdm(total=queue.qsize()) as pbar:
    lock = pbar.get_lock()
    pool.map(process_page_wrapper, range(pool._processes))

pool.join()

100%|██████████| 28026/28026 [20:00<00:00, 23.35it/s] 


In [30]:
for filename in tqdm(filenames):
    get_keywords(filename)

 70%|██████▉   | 19583/28026 [21:05<09:05, 15.48it/s]  


OSError: [Errno 28] No space left on device: 'keywords/5217.dat_kw.txt'